In [ ]:
import torch
class DialogueTemplate():
    system_token = "<|system|>" #system prompt
    user_token = "<|user|>"
    assistant_token = "<|assistant|>"
    end_token = "<|end|>"

    messages = []

    # def __init__(self, tokenizer):
    #     self.tokenizer = tokenizer

    def get_special_tokens(self):
        return [self.system_token,self.user_token, self.assistant_token, self.end_token]

    # system_token: system_prompt <end>
    # user_token: user_msg_1 <end>
    # assistant model_ans_1 <end>
    # user........
    # <|assistant|> for inference (add during inference so model pretends to be an assistant)

    def get_training_prompt(self, infer = False):
        keys = ["product_name","price","warranty","refundable", "inventory","dimensions","reviews", "description"]
        # keys = ["order_status","product_name","refundable","days_since_ordered"]
        sys_string = ""
        for key in keys:
          if key in self.message.keys() and len(str(self.message[key])) > 0:
            sys_string += key + ": " + str(self.message[key]) + "\n"
          # else:
            # sys_string += key + ': ""\n'
        prompt = self.system_token + "\n" + sys_string  + self.end_token + "\n"
        prompt += self.user_token + "\n" + self.message["user_question"] + self.end_token + "\n"
        try:
            prompt += self.assistant_token + "\n" + self.message["support_answer"] + self.end_token + "\n"
        except:
            pass
            # elif message["role"] == "system":
            #     prompt += self.system_token + "\n" + message["value"] + self.end_token + "\n"
        if infer == False:
          prompt += '<|endoftext|>'
        # prompt = prompt +  ' <|endoftext|>'
        return prompt #fully formed tranining prompt

    def get_inference_prompt(self):
        prompt = self.get_training_prompt(infer=True)
        prompt += self.assistant_token
        return prompt

    def get_raw_dialogue(self):
        prompt = ''
        for message in self.messages:
            prompt += message["content"] + '\n'

        return prompt

    def prepare_dialogue(self, example):
        if 'content' in example.keys() and example['content'] is not None:
            self.messages = example['content']
        else:
            # print("Invalid conversations")
            pass

        self.message = example

        example['text'] = self.get_training_prompt()
        return example

def get_dialogue_template():
    return DialogueTemplate()

examples = {
        "product_name":"Compact Microwave",
        "price":"",
        "warranty":"",
        "refundable":"returnable within 20 days",
        "inventory":"",
        "dimensions":"",
        "reviews":"",
        "description":"",
        "user_question":"I'd like to cancel my compact microwave order that was placed a while ago. Can you assist?",
        "support_answer":"The order is within the 20-day return period. I'll proceed with the cancellation now. cancel_order('Compact Microwave')",
        "order_status":"ordered",
        "days_since_ordered":10.0
    }
dialogue_template = get_dialogue_template()
dialogue_template.message = examples
print("Training:",dialogue_template.get_training_prompt())
print("Infering:", dialogue_template.get_inference_prompt())


Training: <|system|>
product_name: Compact Microwave
refundable: returnable within 20 days
<|end|>
<|user|>
I'd like to cancel my compact microwave order that was placed a while ago. Can you assist?<|end|>
<|assistant|>
The order is within the 20-day return period. I'll proceed with the cancellation now. cancel_order('Compact Microwave')<|end|>
<|endoftext|>
Infering: <|system|>
product_name: Compact Microwave
refundable: returnable within 20 days
<|end|>
<|user|>
I'd like to cancel my compact microwave order that was placed a while ago. Can you assist?<|end|>
<|assistant|>
The order is within the 20-day return period. I'll proceed with the cancellation now. cancel_order('Compact Microwave')<|end|>
<|assistant|>


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_file = r"train.json"
access_token = HF_TOKEN

In [ ]:
from datasets import load_dataset

# ds = load_dataset("bitext/Bitext-customer-support-llm-chatbot-training-dataset",token=access_token)
# ds['train'].to_json("ds_train_data.json")

Creating json from Arrow format:   0%|          | 0/27 [00:00<?, ?ba/s]

20961372

In [ ]:
import json
f = open("ds_train_data.json","r")
ts = open(data_file,"w")
ts.write('[')
ts.flush()
first = 1
cnt = 0
while True:
  s = f.readline()
  if not s:
    break
  # if (len(s.split(' '))) > 1024:
  #   continue
  if not first:
    ts.write(',')
  else:
    first = 0
  ts.write(s)
  cnt += 1
  if cnt == 2000:
    break
  print(len(s.split(' ')))
ts.write(']\n')
ts.flush()

In [ ]:
import torch
from torch.utils.data import Dataset
from random import shuffle

# from dialogue import get_dialogue_template

class  DialogueDataset(Dataset):
    def __init__(self, data, tokenizer, max_samples, seq_length=1024):
        super().__init__()

        self.data = data
        self.tokenizer = tokenizer
        self.seq_length = seq_length
        self.dialogue_template = get_dialogue_template()

        self.shuffle()
        print("DATA LENGTH: ", len(data))
        if max_samples > 0:
            self.data = self.data[:max_samples]

        # dialogue_tokens = self.dialogue_template.get_special_tokens()
        # print("dialogue tokens are: ",dialogue_tokens)
        # self.tokenizer.add_special_tokens({"additional_special_tokens": dialogue_tokens})
        # self.tokenizer.add_special_tokens({'pad_token':'<pad>'})

    def shuffle(self):
        return shuffle(self.data)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        sample_dialogue = self.dialogue_template.prepare_dialogue(sample)

        input = self.tokenizer(sample_dialogue['text'], return_token_type_ids=False,max_length=self.seq_length,truncation=True,padding='max_length',return_tensors='pt')

        input['input_ids'] = input['input_ids'].squeeze(0)
        input['attention_mask'] = input['attention_mask'].squeeze(0)
        # labels = torch.empty_like(input['input_ids']).copy_(input["input_ids"])
        labels = torch.empty_like(input['input_ids']).copy_(input["input_ids"])

        labels = self.mask_user_labels(labels)

        # labels = torch.cat([labels[1:], torch.full((1,), self.tokenizer.encode('<pad>')[0],dtype=labels.dtype)],dim=0)

        return {
            "input_ids" : input["input_ids"],
            "attention_mask" :input["attention_mask"],
            "labels":labels,
        }

    def mask_user_labels(self, labels):
            user_token_id = self.tokenizer.convert_tokens_to_ids(self.dialogue_template.user_token)
            system_token_id = self.tokenizer.convert_tokens_to_ids(self.dialogue_template.system_token)
            assistant_token_id = self.tokenizer.convert_tokens_to_ids(self.dialogue_template.assistant_token)
            pad_token_id = self.tokenizer.convert_tokens_to_ids('<|pad|>')

            masking = False
            for idx,label_id in enumerate(labels):
                if label_id in [user_token_id, system_token_id]:
                    masking = True #Start masking when user token is encountered
                elif label_id == assistant_token_id:
                    masking = False #Stop masking when assistant token is encountered
                if masking:
                    labels[idx] = pad_token_id #Mask current token if in masking mode
            return labels


def get_dialogue_dataset(data, tokenizer, max_samples,seq_length):
    return DialogueDataset(data, tokenizer, max_samples, seq_length)

In [ ]:
import os
import json
import torch
from torch.utils.data import DataLoader, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup
from tqdm import tqdm

def load_from_file(filename):
    with open(filename, 'r', encoding='utf-8',errors="ignore") as f:
        return json.load(f)

def decode_masked_labels(labels, tokenizer):
    # Replace -100 with tokenizer.pad_token_id
    pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
    labels = torch.where(labels != -100, labels, torch.tensor(pad_id))
    # Decode the labels
    return tokenizer.decode(labels)

model_name = "gpt2-medium"
train_batch_size = 8
test_batch_size = 1
seq_len = 1024

def get_ds(model_name, data_filename):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    # if tokenizer.pad_token is None:
    #     tokenizer.pad_token = tokenizer.eos_token
    dialogue_template = get_dialogue_template()
    special_tokens = dialogue_template.get_special_tokens()

    tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})
    tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
    raw_dataset = load_from_file(data_filename)
    dataset = get_dialogue_dataset(raw_dataset, tokenizer, 30000, seq_len)

    # with open(data_filename, 'r', encoding='utf-8',errors="ignore") as f:
    #     text = f.read()
    # if len(text) > 10_000_000:
    #     text = text[:10_000_000]
    # print("Length of text is:",len(text))

    # tokenized_text = tokenizer.encode(text)
    # seq_length = 1024

    # # Pad tokenized_text to be a multiple of seq_length
    # padding_length = seq_length - (len(tokenized_text) % seq_length)
    # if padding_length < seq_length:  # Only add padding if there is some remainder
    #     tokenized_text.extend([tokenizer.pad_token_id] * padding_length)

    # input_ids = [tokenized_text[i:i+seq_length] for i in range(0, len(tokenized_text), seq_length)]

    # # Create attention masks
    # attention_masks = [[1 if token != tokenizer.pad_token_id else 0 for token in seq] for seq in input_ids]

    # # Create dataset (labels are the same as input_ids)
    # dataset = [{"input_ids": torch.tensor(ids),
    #             "attention_mask": torch.tensor(mask),
    #             "labels": torch.tensor(ids)}  # labels are the same as input_ids
    #         for ids, mask in zip(input_ids, attention_masks)]

    val_size = int(0.1 * len(dataset))
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, pin_memory=True)
    val_dataloader = DataLoader(val_dataset, batch_size=test_batch_size, shuffle=True)

    # for batch in val_dataloader:
    #   if torch.isnan(batch["input_ids"]).any() or torch.isnan(batch["labels"]).any():
    #       print("NaN found in validation data")
    #       break
    # pad_token_id = tokenizer.pad_token_id
    # for batch in tqdm(val_dataloader):
    #   input_ids = batch["input_ids"].to('cuda')
    #   attention_mask = batch["attention_mask"].to('cuda')
    #   labels = batch["labels"].to('cuda')
    #   if torch.isnan(batch["input_ids"]).any() or torch.isnan(batch["labels"]).any():
    #     print("NaN found in validation data")

      # labels[labels == pad_token_id] = -100

    return train_dataloader, val_dataloader, tokenizer

from collections import Counter
def analyze_invalid_tokens(dataloader, tokenizer):
    invalid_token_counter = Counter()
    total_tokens = 0

    for batch in tqdm(dataloader, desc="Analyzing invalid tokens"):
        labels = batch["labels"]
        invalid_ids = (labels < 0) | (labels >= len(tokenizer))
        invalid_token_counter.update(labels[invalid_ids].tolist())
        total_tokens += labels.numel()

    print(f"Total tokens: {total_tokens}")
    # print(f"Total invalid tokens: {sum(invalid_token_counter.values())}")
    print(f"Unique invalid tokens: {len(invalid_token_counter)}")
    print("Top 10 invalid tokens:")
    for token, count in invalid_token_counter.most_common(10):
        token_str = tokenizer.decode([token]) if token >= len(tokenizer) else ""
        print(f"Token ID {token}: {token_str} - {count} occurrences")

    return invalid_token_counter
# data_filename = r"G:\Coding\gpt2\datasets\internet_archive_scifi_v3.txt"
# data_filename = r"/content/gdrive/MyDrive/GPT2-Finetuning/finetune_chatbot/finetune_chatbot/data/oasst.json"
# data_filename = data_file
train_dataloader, val_dataloader, tokenizer = get_ds(model_name, data_file)
for batch in val_dataloader:
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]

    labels=batch["labels"]

    print("input_ids: ", tokenizer.decode(input_ids[0]))
    print("labels: ", decode_masked_labels(labels[0], tokenizer))
    print("attention_mask: ", attention_mask[0])

    break
# analyze_invalid_tokens(val_dataloader, tokenizer)

print(tokenizer.additional_special_tokens)
special_token_ids = tokenizer.convert_tokens_to_ids(tokenizer.additional_special_tokens)
print("vocab size: ", len(tokenizer.vocab), " ids: ", special_token_ids)
tokenized_size = []
lines = 0
sum =  0
for line in open(data_file, 'r'):
    tokenized_input = tokenizer.tokenize(line)
    tokenized_size.append(len(tokenized_input))
    lines += 1
    if lines > 500:
        break
# print(tokenized_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

DATA LENGTH:  1253
input_ids:  <|system|>
product_name: Indoor Air Purifier
price: $229.99
warranty: 2 years
refundable: Yes, within 30 days
inventory: 80
dimensions: 15x15x30 inches
reviews: 4.8 stars (200 reviews)
description: This Indoor Air Purifier features a HEPA filter and multiple purification modes. It includes a digital display and is designed for large rooms to improve air quality.
<|end|>
<|user|>
What purification modes are available on the Indoor Air Purifier, and how does the HEPA filter work?<|end|>
<|assistant|>
The description mentions multiple purification modes and a HEPA filter but does not specify the modes or the details of how the HEPA filter works.<|end|>
<|endoftext|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pad|><|pa

In [ ]:
def finetune_gpt2(train_dataloader, val_dataloader, tokenizer, model, epochs=4, learning_rate=5e-5, warmup_steps=1000, max_grad_norm=1.0, device="cuda"):
    # tokenizer = AutoTokenizer.from_pretrained(model_name)
    model.resize_token_embeddings(len(tokenizer))

    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
    pad_token_id = tokenizer.pad_token_id

    for epoch in range(epochs):
        total_loss = 0
        model.train()
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            labels[labels == pad_token_id] = -100

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss

            total_loss += loss.item()

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

        avg_train_loss = total_loss / len(train_dataloader)
        print(f"Average train loss: {avg_train_loss:.4f}")

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in tqdm(val_dataloader, desc="Validation"):
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                labels = batch["labels"].to(device)

                labels[labels == pad_token_id] = -100
                # for i in range(labels.size(0)):  # Iterate over each sequence in the batch
                #   labels[i, labels[i] == pad_token_id] = -100
                if (labels == -100).sum().item() == seq_len:
                  # print("All padding tokens in val dataset")
                  continue

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                # if torch.isnan(outputs.loss) or torch.isinf(outputs.loss):
                #   print("NaN or Inf detected in loss")
                #   continue  # or handle it appropriately
                loss = outputs.loss
                # if torch.isnan(loss):
                #   print("\nNaN detected in loss calculation\n")
                #   print("Num -100: ", (labels == -100).sum().item())
                #   print("input_ids: ", tokenizer.decode(input_ids[0]))
                #   print("labels: ", labels[0])
                #   print("attention_mask: ", attention_mask[0])

                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_dataloader)
        print(f"Validation loss: {avg_val_loss:.4f}")
        # model.save_pretrained("/content/drive/MyDrive/local_{}_checkpoint".format(model_name,epoch))
        # tokenizer.save_pretrained("/content/drive/MyDrive/local_{}_checkpoint".format(model_name,epoch))

    return model, tokenizer

model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
# model.gradient_checkpointing_enable()
# Fine-tune the model
model, tokenizer = finetune_gpt2(train_dataloader, val_dataloader, tokenizer,model, device="cuda")

# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/ecom_bot_prod")
tokenizer.save_pretrained("/content/drive/MyDrive/ecom_bot_prod")
# from google.colab import runtime
# runtime.unassign()

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Epoch 1/4: 100%|██████████| 141/141 [03:08<00:00,  1.33s/it]


Average train loss: 28.0439


Validation: 100%|██████████| 125/125 [00:10<00:00, 12.22it/s]


...

Epoch 4/4: 100%|██████████| 141/141 [03:06<00:00,  1.32s/it]


Average train loss: 0.4936


Validation: 100%|██████████| 125/125 [00:10<00:00, 12.14it/s]


Validation loss: 0.4886


('/content/drive/MyDrive/ecom_bot_prod/tokenizer_config.json',
 '/content/drive/MyDrive/ecom_bot_prod/special_tokens_map.json',
 '/content/drive/MyDrive/ecom_bot_prod/vocab.json',
 '/content/drive/MyDrive/ecom_bot_prod/merges.txt',
 '/content/drive/MyDrive/ecom_bot_prod/added_tokens.json',
 '/content/drive/MyDrive/ecom_bot_prod/tokenizer.json')

In [ ]:
import os
import json
import torch
from torch.utils.data import DataLoader, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup
from tqdm import tqdm
# modelname = r"gpt2"
modelname = r"/content/drive/MyDrive/simple_customer_bot_mixed"
model = AutoModelForCausalLM.from_pretrained(modelname).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(modelname)
# tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def inference(model, tokenizer, prompt,max_new_tokens=128, temperature=0.3, top_k=100,top_p=0.95):
    model.eval()
    dialogue_template = get_dialogue_template()
    device = "cuda"
    # system_prompt = "[{\"name\": \"is_prime\", \"description\": \"Checks if a number is prime.\", \"parameters\": {\"num\": {\"description\": \"The number to be checked.\", \"type\": \"int\"}}},\n{\"name\": \"get_location\", \"description\": \"Get user's current location.\", \"parameters\": {\"user\": {\"description\": \"Name of user.\", \"type\": \"string\"}}}]"
    # dialogue_template.messages = [{"from":"system","value":system_prompt},{"from": "user", "value": prompt}]
    dialogue_template.message = prompt
    input_text = dialogue_template.get_inference_prompt()
    # print("the input prompt is: ", input_text)
    inputs = tokenizer(input_text, return_tensors="pt",padding=True, truncation=True,max_length=1024).to(device)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        do_sample=True,
        pad_token_id=tokenizer.pad_token_id
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=False)
    # return generated_text
    response = generated_text.split(dialogue_template.assistant_token)[1].strip()
    response = response.replace(dialogue_template.end_token, "").strip()
    # response = response.replace("<pad>", "").strip()
    return response

In [ ]:
prompts = ["What is the cost of the fancy house plant?","Is it better indoor or outdoor?", "How much water does the plant need?", "Can it grow without soil?", "Does this plant need light?", "How many of these plants are in inventory?", "Is the pot self watering?"]
for prompt in prompts:
  print(prompt)
  q = {
        "product_name": "Fancy house plant",
        "price": "$2989.99",
        "warranty": "3-month warranty",
        "refundable": "Yes, within 144 days",
        "inventory": 940,
        "dimensions": "10x12 inches",
        "reviews": "4.9 stars (50 reviews)",
        "description": "Money plant is an excellent indoor plant due to its ability to survive in low light . It is an Air Purifier plant as it efficiently removes indoor pollutants. It is also an easy to care for plant suitable in living room, balcony, bedroom or in hanging baskets. This plant with its beautiful heart shaped leaves with specles of cream or white markings is believed to bring good luck & prosperity to your home according to Feng Shui. Watering requirement for the plant is generally twice a week. But it is ideal to water it whenever the top layer of the soil feels dry. Pot is self watering where excess water gets stored in the reservoir below, the roots then absorbs water via capillary action and the plants get water as and when required, allowing you to water less frequently. Making it perfect plant for office desk.",
        "user_question": prompt
  }
  generated_text = inference(model, tokenizer, q,top_k=50,temperature=0.3,top_p=1.0,max_new_tokens=256)
  print(f"{generated_text}")

What is the cost of the fancy house plant?
The price of the Fancy house Plant is $2989.99.
<|endoftext|>
Is it better indoor or outdoor?
The description mentions that it is ideal for indoor or outdoor conditions but doesn't specify if it's better or worse. It focuses on its versatility and air purifier properties but doesn't provide detailed recommendations on what types of plants might work best for different indoor/outdoor conditions.
<|endoftext|>
How much water does the plant need?
The description specifies that the Fancy houseplant needs around 2-3x a week of water.
<|endoftext|>
Can it grow without soil?
Yes, the description mentions that the Fancy house plant has a medium-sized, multi-leafed structure. It has a small
<|endoftext|>
Does this plant need light?
The description doesn't mention if it needs light or if it needs rain. It focuses on the ability of the plant to clean and produce good drainage.
<|endoftext|>
How many of these plants are in inventory?
We have 940 of Fancy 

In [ ]:
prompts = ["Is the suit made of cotton?", "Is it water proof?", "What is the price for the swimsuit?", "What is the refund policy for the swimsuit?", "Is there a warranty on the Evening Gown?",
           "How many mouse are in stock","What are the reviews for the Gown?", "How many buttons does the mouse have?", "What is the refund policy?"]
for prompt in prompts:
  print(prompt)
  q = {
        "product_name": "Dell Mouse",
        "price": "$299.99",
        "warranty": "3-month warranty",
        "refundable": "Yes, within 144 days",
        "inventory": 40,
        "dimensions": "10x12 inches",
        "reviews": "4.9 stars (50 reviews)",
        "description": "Designed for comfortable use over extended periods of time, the Dell Optical Mouse is your desktop partner. Shaped and sized to fit the contours of your hand with two buttons and a scroll wheel, the Dell Optical Mouse makes it easy to navigate through on-screen projects. ",
        "user_question": prompt
  }
  generated_text = inference(model, tokenizer, q,top_k=40,temperature=0.3,top_p=1.0,max_new_tokens=256)
  print(f"{generated_text}")

Is the suit made of cotton?
The description does not specify if the suit is made of cotton or polyester. It focuses on the optical mouse's ergonomic design and comfort features.
<|endoftext|>
Is it water proof?
The description doesn't specify if the Dell Optical Mouse is waterproof, but it does mention that it's designed for extended use. The description doesn't mention if it's water-resistant, so it's likely waterproof. The description doesn't mention if it's water-resistant with a scroll wheel, so it's likely not waterproof. The description doesn't mention if it's waterproof with a scroll wheel with two buttons, so it's likely not waterproof. The description doesn't mention if it's waterproof with a scroll wheel with two buttons with a scroll wheel, so it's likely not waterproof. The description doesn't mention if it's waterproof with a scroll wheel with two buttons with a scroll wheel, so it's likely not waterproof. The description doesn't mention if it's waterproof with a scroll wh

In [ ]:
prompts = ["I would like to cancel my order for the watch.", "What is the price of the swiss watch?", "Is the swiss watch in stock?", "What is the refund policy for the watch?"]
for prompt in prompts:
  print(prompt)
  q = {
  "product_name":"Swiss Watch",
  "price": "$55000",
  "warranty":"No warranty",
  "refundable": "Yes, within 200 days",
  "inventory": "2",
  "dimensions": "XL",
  "order_status":"delivered",
  "days_since_ordered":15,
  "reviews": "500 reviews - 4.6 stars",
  "description" : "Best quality swiss watch. Waterproof. Lasts 100 years.",
  "user_question":prompt}
  generated_text = inference(model, tokenizer, q,top_k=40,temperature=0.3,top_p=1.0,max_new_tokens=256)
  print(f"Generated text:\n{generated_text}")

I would like to cancel my order for the watch.
Generated text:
Yes, you can cancel your order within 200 days.
<|endoftext|>
What is the price of the swiss watch?
Generated text:
The price of the Swiss Watch is $55000.
<|endoftext|>
Is the swiss watch in stock?
Generated text:
Yes, the Swiss Watch is in stock.
<|endoftext|>
What is the refund policy for the watch?
Generated text:
The refund policy for the Swiss Watch is within 200 days.
<|endoftext|>
